In [ ]:
import requests
import time
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
from h5py import File

## Select Cruises from the official catalog
For this we need to access the full catalog of cruises of the Healy on [Rolling Deck to Repository (R2R)](http://www.rvdata.us/catalog/Healy) first. With all the found links to cruises on the webpage we collect the r2rnav files for each cruise and put time, longitude, latitude, speed and heading into a central DataFrame. We also create a column for the images. This column will be left empty (NaN). For the purpose of this research we only need the points around h:00:50 and h:01:09 for every hour h [1]. The filtering is done by regex. Finally, a column for the ice concentration is added and filled with 254 (_unknown_).

In [ ]:
cruises = requests.get("http://www.rvdata.us/catalog/Healy")
cruises = BeautifulSoup(cruises.text, features='lxml')

In [ ]:
base_url = "http://get.rvdata.us/cruise/{0}/products/r2rnav/{0}_1min.r2rnav"
comp_re = re.compile("T\d{2}\:(01\:0[0-9]|00\:5[0-9])")
locations = None
i = 0

for link in cruises.select("a[href*=catalog/]"):
    
    # check if after 2009
    if int(link.text[3:5]) >= 10:
        print(base_url.format(link.text))
        locations_tmp = pd.read_table(base_url.format(link.text), header=None, 
                                      names=["time", "longitude", "latitude", "speed", "heading", "image"], 
                                      index_col=0, skiprows=3)
        
        if not locations_tmp.empty:
            if locations is not None:
                locations = locations.append(locations_tmp[locations_tmp.index.str.contains(comp_re)])
            else:
                locations = locations_tmp[locations_tmp.index.str.contains(comp_re)].copy()
    
locations.index = pd.to_datetime(locations.index)
locations["ice"] = 254 

## Collect images
The images are taken at h:01 for every hour h. With the filtered time we can now build a request to download the image and save it in the DataFrame. There are two possibility provided. The first one downloads all the images and saves the bytes into the DataFrame. The second one builds a list of URLs, which can be downloaded by an external tool, f.e. wget, and saves only the filesystem paths to the images to the DataFrame.  
**Caution:** You need to make sure that your folder structure, the downloaded images and the paths in the DataFrame are consistent.

In [ ]:
# direct download
base_url_img = "http://icefloe.net/Aloftcon_Photos/albums/{}/{}.jpeg"

for point in locations.index:

    count = point.strftime("%Y%m%d-%H01")

    img = requests.get(base_url_img.format(point.year, count))

    locations.loc[point, "image"] = img.content
    time.sleep(1)

In [ ]:
# prepare list of urls for download, add paths to DataFrame
base_url_img = "http://icefloe.net/Aloftcon_Photos/albums/{}/{}.jpeg"
base_path = "data/images/remote/{}.jpeg"

with open("data/download/url_list_img", "w") as url_list:
    for point in locations.index:

        count = point.strftime("%Y%m%d-%H01")
        
        url_list.write(base_url_img.format(point.year, count) + "\n")
        
locations["image"] = locations.index.map(lambda x: base_path.format(x.strftime("%Y%m%d-%H01")))

In [ ]:
locations

## Ice concentration
With all the geographical information (place and time) about the images, we now can label the images with the ice extent data from the [Near-Real-Time SSM/I-SSMIS EASE-Grid Daily Global Ice Concentration and Snow Extent](http://nsidc.org/data/docs/daac/nise1_nise.gd.html) project (NISE) [2]. There we find not only the sea ice concentration, but also labels for open water and costal regions. 

First, we need to define a distance measure for finding the closest point to the coordinates of the image in the extent data. I've used the [Haversine formula](https://en.wikipedia.org/wiki/Haversine_formula). Other measures are available.

Additionally, we need the grid definitions for the EASE 2.0 grid to map longitude and latitude to the grid and vice versa. Those can be found on the webpage of the University of Colorado [3]. The 25km-grid is used.

Finally, the algorithm can search the extent for a specific day and position in the NISE data and label every image from that day with the ice extent. Therefore it searches for the closest point in the data based on the coordinates of the image.

I've provided a routine for collecting a list of NISE data based on the dates in the DataFrame index. Be careful, these are still 1,5GB in total.

In [ ]:
def haversine(lons, lats, lon2, lat2):
    lons, lats, lon2, lat2 = map(np.radians, [lons, lats, lon2, lat2])

    dlon = lon2 - lons
    dlat = lat2 - lats

    a = np.sin(dlat/2.0)**2 + np.cos(lats) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    
    idx = np.unravel_index(np.argmin(km), lats.shape)
    return idx

In [ ]:
dates = np.unique(locations.index.date)
base_url_extent = "ftp://n5eil01u.ecs.nsidc.org/SAN/OTHR/NISE.004/{}/NISE_SSMISF17_{}.HDFEOS"

with open("data/download/url_list_extent", "w") as url_list:
    for date in dates:
        folder = date.strftime("%Y.%m.%d")
        file = date.strftime("%Y%m%d")
        url_list.write(base_url_extent.format(folder, file)+"\n")

In [ ]:
lats = np.fromfile('data/extent/grid/EASE2_N25km.lats.720x720x1.double', dtype=np.float64).reshape((720,720))
lons = np.fromfile('data/extent/grid/EASE2_N25km.lons.720x720x1.double', dtype=np.float64).reshape((720,720))

In [ ]:
# missing values for 2012-02-01, nise_ssmisf17_20120201.h5
locations = locations.drop(locations.index[locations.index.date == dates[364]])
dates = np.unique(locations.index.date)

for date in dates:
    file = date.strftime("%Y%m%d")
    
    extent = h5py.File("data/extent/remote/NISE_SSMISF17_{}.h5".format(file), 'r')
    extent = extent["Northern Hemisphere/Data Fields/Extent"]
    
    locations.loc[locations.index.date == date, "ice"] = locations[locations.index.date == date].apply(lambda x: extent[haversine_np(lons, lats, x["longitude"], x["latitude"])], axis=1)

In [ ]:
locations.to_hdf("data/backup/complete_paths.h5", "data")

## References
[1] Because the timestamps of the pictures are always one minute after every full hour.

[2] Nolin, A., R. L. Armstrong, and J. Maslanik. 1998. Near-Real-Time SSM/I-SSMIS EASE-Grid Daily Global Ice Concentration and Snow Extent. Version 4. ice extent. Boulder, Colorado USA: NASA DAAC at the National Snow and Ice Data Center. 

[3] ftp://sidads.colorado.edu/pub/tools/easegrid2/